Combustion around a square cylinder using a finite reaction model. 

In [ ]:
#r "D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

## Open Database

In [ ]:
BoSSSshell.WorkflowMgm.Init("Combustion_HeatedSquareCylinder");


Project name is set to 'Combustion_HeatedSquareCylinder'.
Opening existing database '\\hpccluster\hpccluster-scratch\gutierrez\Combustion_HeatedSquareCylinder'.


In [ ]:
static var myBatch = GetDefaultQueue();
static var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;
// static var myDb = OpenOrCreateDatabase(@"\\fdygitrunner\ValidationTests\CounterFlowFlame");

In [ ]:
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation()

## Case configuration

In [ ]:
int[] Resolutions = new int[]{3}; 
int[] DGdegree = new int[]{3};
int[] nCells  =  Resolutions.Select(r => (int)(Math.Pow(2,r+1))).ToArray();
double[] Reynolds = new double[] {50}; // 100 is unsteady
int dgMF = 3;

## Create grid

In [ ]:
public static class GridFactory {
    public static double[] GetRNodes(int Res) { 
//        var rNodes = Grid1D.ExponentialSpaceing(0.5, 29.5, Res + 1, 1.2);
        // var rNodes = Grid1D.TanhSpacing(0.5, 29.5, Res + 1, 2, true);
        var rNodes = Grid1D.TanhSpacing(0.5, 15.5, Res + 1, 2, true);
        return rNodes;
    }
 
    static double[] GetSNodes(int Res) {
        var sNodes =  Grid1D.ExponentialSpaceing(0.0, 1.0, Res + 1, 1.0);
        return sNodes;
    }
    
    public static double[] GetXNodes(int Res) { 
        var xNodes = GenericBlas.SinLinSpacing(-4.0, 22.0, 0, 3*Res + 1);
        return xNodes;
    }
 
    static double[] GetYNodes(int Res) {
        double[] yNodes =  GenericBlas.SinLinSpacing(-8.0,8.0, 0, 2*Res + 1);
        return yNodes;
    }
    
    public static Grid2D GenerateGrid_curved(int Res, int dg) { 
        var rNodes = GetRNodes(Res);
        var sNodes = GetSNodes(Res);
        Grid2D grd;
        if(dg == 1){
             grd = Grid2D.CurvedSquareGrid(rNodes, sNodes, CellType.Square_4, true);
        } else if(dg == 2){
             grd = Grid2D.CurvedSquareGrid(rNodes, sNodes, CellType.Square_9, true);
        } else {
             grd = Grid2D.CurvedSquareGrid(rNodes, sNodes, CellType.Square_16, true);
        }
        
      
        grd.EdgeTagNames.Add(1, "Velocity_inlet");
        grd.EdgeTagNames.Add(2, "Pressure_outlet");
        grd.EdgeTagNames.Add(3, "Velocity_inlet_Cylinder");

        // grd.EdgeTagNames.Add(3, "Wall_tempfixed_hot");
        grd.DefineEdgeTags( delegate (double[] X) {
            double x = X[0];
            double y = X[1];

            // left velocity inlet
            if ((x * x + y * y) > 2 && x <= 0)
                return 1;

            //right pressure outlet
            if ((x * x + y * y) > 2 && x >  0)
                return 2;

            else return 3;
        });
        
             
        bool force = true; 
        myDb.SaveGrid(ref grd, force);
        
        return grd;
     }
 
      
    public static Grid2D GenerateGrid(int Res) { 
        double[] CutOut1Point1 = new double[2] { -0.5, -0.5 }; 
        double[] CutOut1Point2 = new double[2] {  0.5,  0.5 };
        
        var CutOut1 = new BoSSS.Platform.Utils.Geom.BoundingBox(2);
        CutOut1.AddPoint(CutOut1Point1);
        CutOut1.AddPoint(CutOut1Point2);
        
        var xNodes = GetXNodes(Res);
        var yNodes = GetYNodes(Res);
        
        var grd    = Grid2D.Cartesian2DGrid(xNodes, yNodes, CutOuts: CutOut1);
        
        grd.EdgeTagNames.Add(1, "Wall_tempfixed_cold");
        grd.EdgeTagNames.Add(2, "Velocity_inlet");
        grd.EdgeTagNames.Add(3, "Pressure_outlet");
        grd.EdgeTagNames.Add(4, "Velocity_inlet_Cylinder");
//        grd.EdgeTagNames.Add(5, "NoSlipNeumann");
        grd.DefineEdgeTags( delegate (double[] X) {
            double x = X[0];
            double y = X[1];

            //right pressure outlet 
            if (Math.Abs(x - 22.0 ) < 1e-8)
                return 3;

            //top wall
            if (Math.Abs(y - 8.0 ) < 1e-8)
                // return 1;
                return 3;


            // left velocity inlet
            if (Math.Abs(x + 4.0 ) < 1e-8)
                return 2;

            //bottom Wall
            if (Math.Abs(y + 8.0 ) < 1e-8)
            // return 1;
            return 3;
            else return 4;
        });
        
        bool force = true; 
        myDb.SaveGrid(ref grd, force);
        
        return grd;
     }
 }

In [ ]:
public static class BoundaryValueFactory { 

    public static string GetPrefixCode(double Th, double Tc, double Froude, double const_val ) {
        using(var stw = new System.IO.StringWriter()) {
           
           stw.WriteLine("static class BoundaryValues {");
           stw.WriteLine("  static public double VelX(double[] X) {");
           stw.WriteLine("    return 6.0 * (0.5 + X[1]) * (0.5 - X[1]);");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double VelY(double[] X) {");
           stw.WriteLine("    return 0.0;");
           stw.WriteLine("  }");

           stw.WriteLine("static public double CylinderTemperature(double[] X, double t) {");
           stw.WriteLine("double T;");
           stw.WriteLine("double TH = 1.0;");
           stw.WriteLine("double TM ="+const_val+ " ;");
           stw.WriteLine("double t1 = 2; ");// time where temperature starts increasing
           stw.WriteLine("double t2 = 5; ");// temperature peak
           stw.WriteLine("double t2_2 = 7; ");// temperature peak
           stw.WriteLine("double t3 = 10; ");// time where temperature is again at initial state
           stw.WriteLine("if (t > t1 && t < t2)");
           stw.WriteLine("T = ((TM - TH) / (t2 - t1)) * (t - t1) + TH;");
           stw.WriteLine("else if (t >= t2 && t < t2_2) {");
           stw.WriteLine("T = TM;");
           stw.WriteLine("} else if (t >= t2_2 && t < t3) {");
           stw.WriteLine("T = ((TM - TH) / (t2 - t3)) * (t - t3) + TH;");
           stw.WriteLine("} else {");
           stw.WriteLine("T = TH;");
           stw.WriteLine("}");
           stw.WriteLine("return T;");
           stw.WriteLine("}");
           

           stw.WriteLine("  static public double CylinderVx(double[] X) {");
           stw.WriteLine("    return X[0]*0.1/0.5;"); // 0.3 hardcoded velocity, 0.5 hardcoded radius
           stw.WriteLine("  }");

           stw.WriteLine("  static public double CylinderVy(double[] X) {");
           stw.WriteLine("    return X[1]*0.1/0.5;"); // 0.3 hardcoded velocity, 0.5 hardcoded radius
           stw.WriteLine("  }");

           stw.WriteLine("  static public double TemperatureHot(double[] X) {");
           stw.WriteLine("    return 1.5;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double TemperatureCold(double[] X) {");
           stw.WriteLine("    return 1.0;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double One(double[] X) {");
           stw.WriteLine("    return 1.0;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double Zero(double[] X) {");
           stw.WriteLine("    return 0.0;");
           stw.WriteLine("  }");
        
           stw.WriteLine("  static public double Temperature(double[] X) {");
           stw.WriteLine("    return (-2.333 * X[1]) + (4.333 / 2);");
           stw.WriteLine("  }");
            
           stw.WriteLine(" static public double InitialVelocityVx(double[] X) { ");
           stw.WriteLine("    return 1.0 - 0 * Convert.ToInt32(((X[0]-2)*(X[0]-2)+X[1]*X[1])<=1) * X[1];");
           stw.WriteLine("    }"); 
            
           stw.WriteLine(" static public double InitialVelocityVy(double[] X) { ");
           stw.WriteLine("    return 0.0 + 0 * Convert.ToInt32(((X[0]-2)*(X[0]-2)+X[1]*X[1])<=1) * (X[0]-2);");
           stw.WriteLine("    }"); 

           stw.WriteLine("  static public double ConstantVal(double[] X) {");
           stw.WriteLine("    return " + const_val+";");
           stw.WriteLine("  }");

           stw.WriteLine(" static public double InitialPressure(double[] X) { ");
            stw.WriteLine("         return (-1)* X[1] / ("+Froude * Froude +") ;");
            stw.WriteLine("    }");
            stw.WriteLine("}"); 
           return stw.ToString();
        }
    }
   
    static public Formula Get_VelX(double Th, double Tc , double Froude) {
        return new Formula("BoundaryValues.VelX", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude, -1));
    }

    static public Formula Get_VelY(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.VelY", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,-1));
    }
    static public Formula Get_TemperatureHot(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.TemperatureHot", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,-1));
    }

     static public Formula Get_TemperatureCold(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.TemperatureCold", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,-1));
    }

    static public Formula Get_One(){
        return new Formula("BoundaryValues.One", AdditionalPrefixCode:GetPrefixCode(-1,-1,-1,-1));
    }
    static public Formula Get_Zero(){
        return new Formula("BoundaryValues.Zero", AdditionalPrefixCode:GetPrefixCode(-1,-1,-1,-1));
    }
    static public Formula Get_Temperature(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.Temperature", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,-1));
    }
    static public Formula Get_InitialPressure(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.InitialPressure", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,-1));
    }
    static public Formula Get_InitialVelocityVx(){
        return new Formula("BoundaryValues.InitialVelocityVx", AdditionalPrefixCode:GetPrefixCode(-1,-1,-1,-1));
    }
    static public Formula Get_InitialVelocityVy(){
        return new Formula("BoundaryValues.InitialVelocityVy", AdditionalPrefixCode:GetPrefixCode(-1,-1,-1,-1));
    }
    static public Formula Get_ConstantVal(double constVal){
        return new Formula("BoundaryValues.ConstantVal", AdditionalPrefixCode:GetPrefixCode(-1,-1,-1,constVal));
    }
    static public Formula Get_CylinderVelocityX(){
        return new Formula("BoundaryValues.CylinderVx", AdditionalPrefixCode:GetPrefixCode(-1,-1,-1,-1));
    }

    static public Formula Get_CylinderVelocityY(){
        return new Formula("BoundaryValues.CylinderVy", AdditionalPrefixCode:GetPrefixCode(-1,-1,-1,-1));
    }


    static public Formula Get_Temperature_Cylinder(double maxTemperature){
        return new Formula("BoundaryValues.CylinderTemperature",TimeDep:true, AdditionalPrefixCode:GetPrefixCode(-1,-1,-1,maxTemperature));
    }
}

## Create base control file
In this ControlFile basic configuration of the CounterDiffusionFlame is defined.

In [ ]:
static XNSEC_Control GiveMeTheCtrlFile(int dg, int Res, bool isMF, double Re) {
    var CC = new ChemicalConstants();

    var C = isMF ? new XNSEC_MF_Control() : new XNSEC_Control();

            C.NumberOfChemicalSpecies = 4;
            C.SetDGdegree(dg);
            var nCells = (int)Math.Pow(2, Res + 1); 

            bool useCurvedMesh = false;
            if(useCurvedMesh){
            C.SetGrid(GridFactory.GenerateGrid_curved(nCells, dg));
            } else{
            C.SetGrid(GridFactory.GenerateGrid(nCells));
            }

         
            C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Res", Res));
            C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Dgdegree", dg));
            C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Reynolds", Re));
            C.SessionName = "Combustion_HeatedCylinder" + Res + "_DG" + dg + "_Re" + Re;


            double TemperatureInFuel = 300;  //
            double TemperatureInOxidizer = 300; //
            double AtmPressure = 101325; // Pa
            double[] FuelInletConcentrations = new double[] { 0.2, 0.0, 0.0, 0.0, 0.8 };

            double[] OxidizerInletConcentrations = new double[] { 0.0, 0.23, 0.0, 0.0, 0.77 }; //
            double[] MWs = new double[] { CC.MW_CH4, CC.MW_O2, CC.MW_CO2, CC.MW_H2O, CC.MW_N2 };

            double mwFuel = CC.getAvgMW(MWs, FuelInletConcentrations);
            double mwAir = CC.getAvgMW(MWs, OxidizerInletConcentrations);
            double densityAirIn = AtmPressure * mwAir / (CC.R_gas * TemperatureInOxidizer * 1000); // kg / m3
            double densityFuelIn = AtmPressure * mwFuel / (CC.R_gas * TemperatureInFuel * 1000); // kg / m3.

            // Reference values
            //===================
            // Basic units to be used: Kg, m, s, mol, pa,
            double TRef = TemperatureInOxidizer;// Reference temperature  is the inlet temperature, (K)
            double pRef = AtmPressure; // Pa
            double uRef = 1.0;// Math.Max(uInFuel, uInAir); // m/s
            double LRef = 1;

            C.YFuelInlet = FuelInletConcentrations[0];
            C.YOxInlet = OxidizerInletConcentrations[1];
            C.FuelInletConcentrations = FuelInletConcentrations;
            C.OxidizerInletConcentrations = OxidizerInletConcentrations;

            double s = (CC.nu_O2 * CC.MW_O2) / (CC.nu_CH4 * CC.MW_CH4);
            C.phi = s * C.YFuelInlet / C.YOxInlet;
            C.zSt = 1.0 / (1.0 + C.phi);
            var MLC = new MaterialLawCombustion(300, new double[] { }, C.MatParamsMode, C.rhoOne, true, 1.0, 1, 1, C.YOxInlet, C.YFuelInlet, C.zSt, CC, 0.75);
            var ThermoProperties = new ThermodynamicalProperties();

            //==========================
            //Derived reference values
            //==========================
            C.uRef = uRef; // Reference velocity
            C.LRef = LRef; // reference length
            C.pRef = AtmPressure; // reference pressure
            C.TRef = TemperatureInFuel;// reference temperature
            C.MWRef = MLC.getAvgMW(MWs, C.OxidizerInletConcentrations); // Air mean molecular weight
            C.rhoRef = C.pRef * C.MWRef / (8.314 * C.TRef * 1000); // Kg/m3. ok ;
            C.cpRef = 1.3;//ThermoProperties.Calculate_Cp_Mixture(new double[] { 0.23, 0.77 }, new string[] { "O2", "N2" }, 300); // 1.219185317353029;// Representative value, KJ/Kg K ========> 1.31 for the one-step kinetic model
            C.muRef = MLC.getViscosityDim(300);
            C.MolarMasses = new double[] { C.CC.MW_CH4, C.CC.MW_O2, C.CC.MW_CO2, C.CC.MW_H2O, C.CC.MW_N2 };
            C.MolarMasses.ScaleV(1.0 / C.MWRef); //NonDimensionalized Molar masses
            C.T_ref_Sutherland = 300;
            double heatRelease_Ref = (C.TRef * C.cpRef);
            C.B = CC.PreExponentialFactor;

            C.StoichiometricCoefficients = new double[] { -1, -2, 1, 2, 0 };
            bool simpleCase = true;

            if (simpleCase) {
                C.Damk = C.rhoRef * C.LRef * C.B / (C.uRef * C.MWRef)*1e4;//*1e-3;
                Console.WriteLine("Damkoehler Number :" + C.Damk);

                C.ReactionRateConstants = new double[] { C.Damk, CC.Ta / TRef, 1.0, 1.0 };
                C.HeatRelease = C.CC.HeatReleaseMass / heatRelease_Ref;
            } else {
                C.Damk = C.rhoRef * C.LRef * C.B / (C.uRef * C.MWRef);
                Console.WriteLine("Damkoehler Number :" + C.Damk);

                C.ReactionRateConstants = new double[] { C.Damk, CC.Ta / TRef, 1.0, 1.0 };
                C.HeatRelease = C.CC.HeatReleaseMass / heatRelease_Ref;
            }

            double maxTemperature =   C.zSt * C.TFuelInlet + (1 - C.zSt) * C.TOxInlet + C.HeatRelease * C.YFuelInlet * C.zSt;
            
            Console.WriteLine("Max flame temperature should be: "+ maxTemperature);
            C.EnableMassFractions = true;
            C.ChemicalReactionActive = true;
            C.MatParamsMode = MaterialParamsMode.Sutherland;
            C.physicsMode = PhysicsMode.Combustion;

            C.smoothingFactor = 80;
            //    C.TimesteppingMode = AppControl._TimesteppingMode.Steady;

            // C.TimesteppingMode = AppControl._TimesteppingMode.Transient;
            // C.TimeSteppingScheme = BoSSS.Solution.XdgTimestepping.TimeSteppingScheme.BDF3;
            // double dt = 0.1;
            // C.dtMax = dt;
            // C.dtMin = dt;
            // C.Endtime = 100;
            // C.NoOfTimesteps = 15000;
            // C.LinearSolver.SolverCode = LinearSolverCode.exp_Kcycle_schwarz;
            // C.LinearSolver.NoOfMultigridLevels = 5;
            // C.LinearSolver = LinearSolverCode.exp_Kcycle_schwarz.GetConfig();

            C.LinearSolver = LinearSolverCode.direct_pardiso.GetConfig();

            C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;

            C.NonLinearSolver.ConvergenceCriterion = 1e-11;
            C.NonLinearSolver.verbose = true;
            C.NonLinearSolver.MaxSolverIterations = 15;

            C.PenaltyViscMomentum = 1.0 * 1;
            C.PenaltyHeatConduction = 1.0 * 1;
            C.PhysicalParameters.IncludeConvection = true;

            C.UseSelfMadeTemporalOperator = true;
            C.timeDerivativeEnergyp0_OK = false;
            C.timeDerivativeConti_OK = true;
            C.EnableTemperature = true;

            if (simpleCase) {
                C.rhoOne = true;
                C.UseSelfMadeTemporalOperator = false;
                C.timeDerivativeEnergyp0_OK = false;
                C.timeDerivativeConti_OK = false;
                C.MatParamsMode = MaterialParamsMode.Constant;
            }

            C.Reynolds = Re;
            C.Prandtl = 0.71;
            double Fr = 11111111; //shouldnt matter

            //    double Fr = Math.Sqrt(2.0 * C.Prandtl * (Th - Tc) / (Th + Tc));
            C.Froude = Fr;
            C.HeatCapacityRatio = 1.4;
            C.T_ref_Sutherland = 600;  //Check
    //==========================
    // Initial conditions
    //==========================


    C.AddInitialValue(VariableNames.VelocityX, BoundaryValueFactory.Get_InitialVelocityVx());
        C.AddInitialValue(VariableNames.VelocityY, BoundaryValueFactory.Get_InitialVelocityVy());
        // C.AddInitialValue(VariableNames.VelocityX, BoundaryValueFactory.Get_ConstantVal(1.0));
        // C.AddInitialValue(VariableNames.VelocityY, BoundaryValueFactory.Get_ConstantVal(0.0));
        C.AddInitialValue(VariableNames.Pressure, BoundaryValueFactory.Get_Zero());

    //==========================
    // Boundary conditions
    //==========================


    C.AddBoundaryValue("Velocity_inlet", VariableNames.VelocityX, BoundaryValueFactory.Get_ConstantVal(1.0));
    C.AddBoundaryValue("Velocity_inlet", VariableNames.VelocityY, BoundaryValueFactory.Get_Zero());
   

    C.AddBoundaryValue("Pressure_outlet", VariableNames.Pressure, BoundaryValueFactory.Get_Zero());

  


    if(useCurvedMesh){
        C.AddBoundaryValue("Velocity_inlet_Cylinder", VariableNames.VelocityX, BoundaryValueFactory.Get_CylinderVelocityX());
        C.AddBoundaryValue("Velocity_inlet_Cylinder", VariableNames.VelocityY, BoundaryValueFactory.Get_CylinderVelocityY());
    } else{
        C.AddBoundaryValue("Velocity_inlet_Cylinder", VariableNames.VelocityX, BoundaryValueFactory.Get_CylinderVelocityX());
        C.AddBoundaryValue("Velocity_inlet_Cylinder", VariableNames.VelocityY, BoundaryValueFactory.Get_CylinderVelocityY());

        // C.AddBoundaryValue("Wall_tempfixed_cold", VariableNames.Temperature, BoundaryValueFactory.Get_One());

    }



    return C;
    }

## Starting the MixtureFraction simulation
Configuration  for the simulation using the mixture fraction approach, where an infinite reaction rate is assumed. Used to find adequate starting solution for the full problem.

In [ ]:
static XNSEC_Control GiveMeTheMixtureFractionCtrlFile(int dg, int nCells, double Re){
    var C_MixtureFraction = GiveMeTheCtrlFile(dg, nCells, true,Re);
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.ProjectName = "CylinderFlame";
    string name = C_MixtureFraction.ProjectName + "P" + dg + "K" + nCells;
    C_MixtureFraction.SessionName = "FS_" + name;

    C_MixtureFraction.UseSelfMadeTemporalOperator = false;
    C_MixtureFraction.ChemicalReactionActive = false;
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.NonLinearSolver.MaxSolverIterations = 50; 
    
    // Boundary and initial conditions
        C_MixtureFraction.AddInitialValue(VariableNames.MixtureFraction,BoundaryValueFactory.Get_ConstantVal(1.0));
    C_MixtureFraction.AddBoundaryValue("Velocity_inlet_Cylinder", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantVal(1.0));
    C_MixtureFraction.AddBoundaryValue("Velocity_inlet", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantVal(0.0));



    bool useHomotopy = false;

    if(useHomotopy) { 
        C_MixtureFraction.HomotopyApproach = XNSEC_Control.HomotopyType.Automatic;
        // C_MixtureFraction.HomotopyVariable = XNSEC_Control.HomotopyVariableEnum.VelocityInletMultiplier;
        // C_MixtureFraction.homotopieAimedValue = multiplier;
        C_MixtureFraction.HomotopyVariable = XNSEC_Control.HomotopyVariableEnum.Reynolds;
        C_MixtureFraction.homotopieAimedValue = C_MixtureFraction.Reynolds;
    }


    C_MixtureFraction.AdaptiveMeshRefinement = true;
    C_MixtureFraction.TimesteppingMode = BoSSS.Solution.Control.AppControl._TimesteppingMode.Steady; 
    int pseudotimesteps = 3;
    C_MixtureFraction.NoOfTimesteps = pseudotimesteps;

    C_MixtureFraction.AMR_startUpSweeps = 3;
   
   double[] p11 = new double[2] {-1.5, -1.5};
   double[] p12 = new double[2] { 4.0,  1.5};   
   var ind1     = new BoSSS.Application.XNSEC.AMRInBoundingBox(p11, p12);
   ind1.maxRefinementLevel = 2;
   C_MixtureFraction.activeAMRlevelIndicators.Add(ind1);
   
   
//    double[] p21 = new double[2] {-1.0, -1.0};
//    double[] p22 = new double[2] { 2.0,  1.0};  
//    var ind2     = new BoSSS.Application.XNSEC.AMRInBoundingBox(p21, p22);
//    ind2.maxRefinementLevel = 2;
//    C_MixtureFraction.activeAMRlevelIndicators.Add(ind2);
   
   
//    double[] p31 = new double[2] { -0.75, -0.75};
//    double[] p32 = new double[2] {  1.50,  0.75};  
//    var ind3     = new BoSSS.Application.XNSEC.AMRInBoundingBox(p31, p32);
//    ind3.maxRefinementLevel = 3;
//    C_MixtureFraction.activeAMRlevelIndicators.Add(ind3);


    // //top left
    // double eps = 0.5;
    // double[] p41 = new double[2] { -0.5-eps , 0.5-eps }; // (-1,0)
    // double[] p42 = new double[2] {  -0.5 +eps , 0.5+eps };   // 0,1)
    // var ind4     = new BoSSS.Application.XNSEC.AMRInBoundingBox(p41, p42);
    // ind4.maxRefinementLevel = 5;
    // C_MixtureFraction.activeAMRlevelIndicators.Add(ind4);



    // //bot left
    // double[] p51 = new double[2] { -0.5-eps , -0.5-eps }; 
    // double[] p52 = new double[2] {  -0.5 +eps , -0.5+eps };  
    // var ind5     = new BoSSS.Application.XNSEC.AMRInBoundingBox(p51, p52);
    // ind4.maxRefinementLevel = 5;
    // C_MixtureFraction.activeAMRlevelIndicators.Add(ind5);










    // C_MixtureFraction.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onProblematicPoints(troubledPoints,C_MixtureFraction.AMR_startUpSweeps) );  
    C_MixtureFraction.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onFlameSheet(C_MixtureFraction.zSt,3) );
    return C_MixtureFraction;
}

## Send and run jobs

In [ ]:

  Type solver_MF = typeof(BoSSS.Application.XNSEC.XNSEC_MixtureFraction);
  var C_MixtureFraction = GiveMeTheMixtureFractionCtrlFile(dgMF, Resolutions[0],Reynolds[0] );
  string jobName       = C_MixtureFraction.SessionName;
  Console.WriteLine(jobName);
  var oneJob           = new Job(jobName, solver_MF);
  oneJob.NumberOfMPIProcs =  8;
  oneJob.SetControlObject(C_MixtureFraction);
    oneJob.Activate(myBatch); 
    // oneJob.Activate(); 





Grid Edge Tags changed.
Damkoehler Number :280307312966081.28
Max flame temperature should be: 6.73711725168036
FS_CylinderFlameP3K3
Deploying job FS_CylinderFlameP3K3 ... 
Deploying executables and additional files ...
Deployment directory: \\hpccluster\hpccluster-scratch\gutierrez\binaries\Combustion_HeatedSquareCylinder-XNSEC2022Apr15_225509
copied 56 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.



In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

All jobs finished.


In [ ]:
myDb.Sessions[0].Export().Do()

Starting export process... Data will be written to the directory: C:\Users\gutierrez\AppData\Local\BoSSS\plots\sessions\Combustion_HeatedSquareCylinder__FS_CylinderFlameP3K3__7bb021f9-ec4f-41af-9717-0a22dd55b6a5


C:\Users\gutierrez\AppData\Local\BoSSS\plots\sessions\Combustion_HeatedSquareCylinder__FS_CylinderFlameP3K3__7bb021f9-ec4f-41af-9717-0a22dd55b6a5

## Starting the finite-rate chemistry simulation

Now that the simulation for an "infinite" reaction rate is done, we use it for initializing the system with finite reaction rate. The goal is to obtain solutions of the counter difussion flame for increasing strain values. We start with a low strain (bigger Dahmkoehler number), which is increased until extintion is (hopefully) found

In [ ]:
static XNSEC_Control GiveMeTheFullCtrlFile(int dg, int nCells,double Re, ISessionInfo SessionToRestart) {
    var C_OneStep = GiveMeTheCtrlFile(dg, nCells, false,Re);
    C_OneStep.physicsMode = PhysicsMode.Combustion;
    C_OneStep.ProjectName = "CylinderFlame";
    string name = C_OneStep.ProjectName + "P" + dg + "K" + nCells;
    C_OneStep.SessionName = "Full_" + name+ "Smooth32" + C_OneStep.smoothingFactor;
    C_OneStep.VariableOneStepParameters = false;


    C_OneStep.UseSelfMadeTemporalOperator = false;
    C_OneStep.myThermalWallType = SIPDiffusionTemperature.ThermalWallType.Adiabatic;
    C_OneStep.Timestepper_LevelSetHandling = BoSSS.Solution.XdgTimestepping.LevelSetHandling.None;
    C_OneStep.UseMixtureFractionsForCombustionInitialization = true;


    C_OneStep.ChemicalReactionActive = true;
    
    C_OneStep.HeatCapacityMode = MaterialLaw_MultipleSpecies.CpCalculationMode.constant;

    bool AMRinEachNewtonStep = false;

    if( AMRinEachNewtonStep) {
        C_OneStep.NoOfTimesteps = 4;
        C_OneStep.NonLinearSolver.MaxSolverIterations = 8; // Do only one newton iteration before refining
        C_OneStep.NonLinearSolver.MinSolverIterations = 8; // Do only one newton iteration before refining
    } else{
        C_OneStep.NoOfTimesteps = 3; // The steady solution will be calculated again and do AMR
        C_OneStep.NonLinearSolver.MaxSolverIterations = 3;
    }



    // limiting of variable values
    Dictionary<string, Tuple<double, double>> Bounds = new Dictionary<string, Tuple<double, double>>();
    double eps = 1e-2;
    Bounds.Add(VariableNames.Temperature, new Tuple<double, double>(1.0 - eps, 10)); // Min temp should be the inlet temperature.
    Bounds.Add(VariableNames.MassFraction0, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4)); // Between 0 and 1 per definition
    Bounds.Add(VariableNames.MassFraction1, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4));
    Bounds.Add(VariableNames.MassFraction2, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4));
    Bounds.Add(VariableNames.MassFraction3, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4));
    // C_OneStep.VariableBounds = Bounds; ///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    // Boundary conditions

    double dummy = 0;
    if(SessionToRestart != null) {
        C_OneStep.SetRestart(SessionToRestart);
    } else {
        C_OneStep.AddInitialValue(VariableNames.Temperature, BoundaryValueFactory.Get_ConstantVal(1.0));
        C_OneStep.AddInitialValue(VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantVal(0.0));
        C_OneStep.AddInitialValue(VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantVal(0.23));
        C_OneStep.AddInitialValue(VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantVal(0.0));
        C_OneStep.AddInitialValue(VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantVal(0.0));
    }
   

    
    C_OneStep.AdaptiveMeshRefinement = true;
    C_OneStep.AMR_startUpSweeps = 5;
    C_OneStep.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onFlameSheet(C_OneStep.zSt,5) );

    // C.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onReactiveZones(3,0.7) );
    C_OneStep.AddBoundaryValue("Velocity_inlet_Cylinder", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantVal(1.0));
    C_OneStep.AddBoundaryValue("Velocity_inlet_Cylinder", VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantVal(C_OneStep.FuelInletConcentrations[0]));
    C_OneStep.AddBoundaryValue("Velocity_inlet_Cylinder", VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantVal(C_OneStep.FuelInletConcentrations[1]));
    C_OneStep.AddBoundaryValue("Velocity_inlet_Cylinder", VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantVal(C_OneStep.FuelInletConcentrations[2]));
    C_OneStep.AddBoundaryValue("Velocity_inlet_Cylinder", VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantVal(C_OneStep.FuelInletConcentrations[3]));


    
    C_OneStep.AddBoundaryValue("Velocity_inlet", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantVal(1.0));
    C_OneStep.AddBoundaryValue("Velocity_inlet", VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantVal(C_OneStep.OxidizerInletConcentrations[0]));
    C_OneStep.AddBoundaryValue("Velocity_inlet", VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantVal(C_OneStep.OxidizerInletConcentrations[1]));
    C_OneStep.AddBoundaryValue("Velocity_inlet", VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantVal(C_OneStep.OxidizerInletConcentrations[2]));
    C_OneStep.AddBoundaryValue("Velocity_inlet", VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantVal(C_OneStep.OxidizerInletConcentrations[3]));



    return C_OneStep;
}

In [ ]:
Type solver = typeof(BoSSS.Application.XNSEC.XNSEC);

In [ ]:
var sess = (myDb.Sessions.Where(s => s.Name == "FS_CylinderFlameP" + 3 + "K" + Resolutions[0])).FirstOrDefault();
var C = GiveMeTheFullCtrlFile(DGdegree[0], Resolutions[0],Reynolds[0], sess);



Grid Edge Tags changed.
Damkoehler Number :280307312966081.28
Max flame temperature should be: 6.73711725168036


In [ ]:
C.zSt

0.22330097087378645

In [ ]:
foreach (int res in Resolutions) {
    foreach (int dg in DGdegree) {
            var sess = (myDb.Sessions.Where(s => s.Name == "FS_CylinderFlameP" + dgMF + "K" + res)).FirstOrDefault();
            var C = GiveMeTheFullCtrlFile(dg, res,Reynolds[0], sess);
            string jobName = C.SessionName;
            Console.WriteLine(jobName);
            var oneJob = new Job(jobName, solver);
            oneJob.NumberOfMPIProcs = 24;
            oneJob.SetControlObject(C);
            oneJob.Activate(); 
        }
    }




Grid Edge Tags changed.
Damkoehler Number :280307312966081.28
Max flame temperature should be: 6.73711725168036
Full_CylinderFlameP3K3Smooth3280
Deploying job Full_CylinderFlameP3K3Smooth3280 ... 
Deploying executables and additional files ...
Deployment directory: \\hpccluster\hpccluster-scratch\gutierrez\binaries\Combustion_HeatedSquareCylinder-XNSEC2022Apr15_230615
copied 56 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.



In [ ]:
var sess = (myDb.Sessions.Where(s => s.Name == "FS_CylinderFlameP" + 2 + "K" + Resolutions[0])).FirstOrDefault();

In [ ]:
// myDb.Sessions[0].Export().Do()

Calculate the full solution for the initial value

In [ ]:
var sess = (myDb.Sessions.Where(s => s.Name == "FS_CylinderFlameP" + 2 + "K" + Resolutions[0])).FirstOrDefault();

In [ ]:
// // wait for all jobs to finish (up to 2 days, check every 10 minutes)
// BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600*24*2), PollingIntervallSeconds:(60*10));

In [ ]:
// // detect failed Jobs in the job management
// var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
//     .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
//                   || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
// suspects

In [ ]:
// NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

In [ ]:
// // Delete all MF calculations
// Console.WriteLine("Deleting mass fraction calculations");
// myDb.Sessions.Where(sess => sess.Name.Contains("FS")).ForEach(sess=> sess.Delete(true))